# Calculating features
The aim of this notebook is to calculate the features of each `.csv` file containing activity. First, each `.csv` is used to calculate features and new `.csv` file with **_features** suffix is created. Then, new column called `activity` is created based on the name of the source file and all rows with empty values. Finally, one file with all features comined, called `merged_features.csv` is created.

In [1]:
from SignalFeatures import *
import pandas as pd
import os

In [6]:
input_directory = '/Users/kasper/Desktop/python/In progress/EngineeringThesis/ML-model/data/processed'

# loop through each .csv file in the directory
for filename in os.listdir(input_directory):
    if filename.endswith('.csv'):
        windowDf = pd.read_csv('/Users/kasper/Desktop/python/In progress/EngineeringThesis/ML-model/data/processed/' + filename)

        # head and tail cut-off seconds
        tail = 1
        head = 1
        windows = []

        # select range based on head and tail seconds
        # adj is used to cut off signal at good point
        range_df = windowDf.iloc[head*100:len(windowDf)-tail*100]
        adj = len(range_df)%500

        # calculate features for window
        for i in range(len(windowDf)-tail*100-adj):
                if i % 500 == 0:    
                    currSeg = range_df.iloc[i:i+501].reset_index(False).drop(columns = ['index'])
                    window_feat = feats_df(currSeg)
                    windows.append(window_feat)

        currDf = pd.concat(windows)

        output_csv = os.path.splitext(filename)[0] + '_features.csv'
        currDf.to_csv(output_csv, index=False)

=====
operands could not be broadcast together with shapes (334,) (335,) 
=====
=====
97 columns passed, passed data had 96 columns
=====
=====
operands could not be broadcast together with shapes (232,) (233,) 
=====
=====
97 columns passed, passed data had 96 columns
=====
=====
97 columns passed, passed data had 94 columns
=====


In [7]:
input_directory = '/Users/kasper/Desktop/python/In progress/EngineeringThesis/ML-model/data/features/'

# func to insert activity label
def check_activity(filename):
    if 'walking' in filename:
        activity = 'walking'
    elif 'downstairs' in filename:
        activity = 'downstairs'
    elif 'upstairs' in filename:
        activity = 'upstairs'
    elif 'lying' in filename:
        activity = 'resting'
    elif 'sitting' in filename:
        activity = 'resting'

    return activity
    
# loop through each .csv file in the directory
# clean indicies, add activity column
for filename in os.listdir(input_directory):
    if filename.endswith('.csv'):
        df = pd.read_csv(input_directory + filename)
        act = check_activity(filename)
        df['activity'] = act
        df.dropna(axis=0)
        df.to_csv(input_directory + filename,index=False)

In [8]:
# get all .csv files and create new df for merging
input_directory = '/Users/kasper/Desktop/python/In progress/EngineeringThesis/ML-model/data/features'
csv_files = [file for file in os.listdir(input_directory) if file.endswith('.csv')]
merged_df = pd.DataFrame()

# loop through each .csv file in the directory
for file in csv_files:
    file_path = os.path.join(input_directory, file)
    df = pd.read_csv(file_path)
    merged_df = pd.concat([merged_df, df], ignore_index=True)

# save all data into .csv file
clean_df = merged_df.dropna()
clean_df.to_csv('merged_features.csv', index=False)